# <font size = 15> 1. Data Cleaning
    
<font size = 4>This is the first step. Cleaning data from the database "raw.db" and creating a table in the "clean.db" database that has all the artist names corrected and with a unique identifier for the same name. We have filtered names that do not coincide their artist_names with the artist_id (given 1 artist_id there were different artist_names non-coinciding) and we have created the longest common string for those cases in which artists_names coincide inside the same artist_id but if looking at substrings (i.e Rihanna, Rihanna feat Eminem,... will be under the same artist_iid (**internal artist_id created to identify uniquely an artist_name**)

In [2]:
# Database
import sqlalchemy as db
import pandas as pd
import sys
import os
import time
import numpy as np
import tqdm

In [189]:
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        print('%2.2f ms' % ((te - ts) * 1000))
        return result
    return timed

def long_substr(data):
    """
    To find the longest common substring in artist name different in same artist-id
    """
    substr = ''
    if len(data) > 1 and len(data[0]) > 0:
        for i in range(len(data[0])):
            for j in range(len(data[0])-i+1):
                if j > len(substr) and all(data[0][i:i+j] in x for x in data):
                    substr = data[0][i:i+j]
    return substr

# 1. Create the list of artists and songs to look for

## 1.1 Connect to the database

In [392]:
#Paths
path_db = os.path.join("..","data","MSD","raw.db")
path_sql_connection_db =  'sqlite:///' + path_db

#Connect
engine = db.create_engine(path_sql_connection_db)
connection = engine.connect()

In [393]:
#@timeit
def query_db(qq, con = connection, to_df = False):
    res = con.execute(qq)
    if to_df:
        return pd.DataFrame(res.fetchall())
    else:
        return res.fetchall()

## 1.2 Get the set of artists

Get a set of all the artists in the DB:

In [218]:
get_artists = query_db("SELECT DISTINCT(artist_id) from YOUTUBE_URL",to_df = True)
get_artists_names = query_db("SELECT DISTINCT(artist_name) from YOUTUBE_URL",to_df = True)
get_artists.columns = ["artist_id"]; get_artists_names.columns = ["artist_name"]

In [219]:
get_artists.shape

(44745, 1)

In [221]:
get_artists_names.shape

(72665, 1)

Convert this set to a list to be able to iterate

In [184]:
sample_artists = list(get_artists["artist_id"])

## 1.3 For one artist get the song names

We do longest common substring intersection. We will save all the artist_id and the common substring intersection in a dictionary to see for all the 44,000 artists, how many can we find a unique name that matches all the registers (artist_name) with that artist_id. To do so we create 2 dictionaries to store the artist_id if we can find a common substring in the artist_names and one dictionary for the deleted artists.

In [251]:
def get_art_names(art_q):
    query_songs = f"SELECT DISTINCT track_id, title, artist_id, artist_name from YOUTUBE_URL where artist_id = '{art_q}'"
    df_songs_art = query_db(query_songs, to_df = True); 
    df_songs_art.columns = ["track_id", "title", "artist_id", "artist_name"];

    # Join the title and the artist name
    df_songs_art["tit&art"] = df_songs_art["title"] + " " + df_songs_art["artist_name"]
    df_songs_art["tit&art"] = df_songs_art["tit&art"].str.lower()
    df_songs_art["title"] = df_songs_art["title"].str.lower()

    #Get artist name (this helps to define the query)
    art_name = list(set(df_songs_art["artist_name"]))
    return art_name

In [252]:
dict_artistId2artistQuery = dict()
dict_artistId2NOTquery = dict()

for art_q in tqdm.notebook.tqdm(sample_artists):
    # Use the function above
    art_name = get_art_names(art_q)

    if len(art_name)>1:

        #In case more than 1 artist name is found for this unique artist_id, find the intersection
        # Find the Longest common substring
        artist_common_name = long_substr(art_name).strip() #avoid white spaces in the first and last position of the LCS

        # If no matchings
        if len(artist_common_name) == 0:
            dict_artistId2NOTquery[art_q] = art_name
            continue
        else:
            dict_artistId2artistQuery[art_q] = artist_common_name
    elif len(art_name) == 1:
        dict_artistId2artistQuery[art_q] = art_name[0]
    else:
        print("Exiting 2")
        break

For each artist in the common longest substring, we have to ensure that the common string is not just 2 letters (i.e artists = ["ABBA", "BANANAS"] then the common substring will be "BA", but "BA" is not in the set of artists!)

In [353]:
# We want to keep the pairs id-name
df_artistname_artistid =  query_db("SELECT DISTINCT artist_id, artist_name from YOUTUBE_URL",to_df = True)
df_artistname_artistid.columns = ["artist_id", "artist_name"]

# Set of the pairs id-name
set_art_artid = set()
for xx in df_artistname_artistid.values:
    set_art_artid.add(tuple(xx))

In [354]:
#Check if that pair is in the database
dict_consensus_artist_names = dict()

# Get the artist_names which coincides in all the artists names of that id
for kk,vv in tqdm.notebook.tqdm(dict_artistId2artistQuery.items()):
    # if the name of the consensus artists is itself an artist name
    if (kk,vv) in set_art_artid: 
        # add its id and the consensus name to the dictionary
        dict_consensus_artist_names[kk] = vv 

**Create the consensus artist_name (the ones for which a single artist_id can identify them)**

In [356]:
df_consensus_artists = pd.DataFrame({"artist_id": np.array(list(dict_consensus_artist_names.keys())),
              "artist_name": np.array(list(dict_consensus_artist_names.values()))})

# Mask:

#Remove all the consensus artists which name is only digits
df_consensus_artists = df_consensus_artists[~df_consensus_artists["artist_name"].str.isnumeric()]

# Remove artist names with a single letter
df_consensus_artists = df_consensus_artists[df_consensus_artists.artist_name.str.len() > 1]

In [357]:
df_consensus_artists.shape

(36180, 2)

# 2. Create an internal identifier

To denote uniquely each artist_name and be able to search only once this name

In [409]:
df_artname_id = df_consensus_artists["artist_name"].drop_duplicates().reset_index()
df_artname_id.columns = ["artist_iid","artist_name"]

# Merge that artist_iid (internal identifier) to the df_consensus_artists
df_consensus_artists_final = pd.merge(df_consensus_artists, df_artname_id, how = "inner", on = "artist_name")

# 3. Join that internal identifier to the full database of MSD

Up to now the main task has been to find artist_id that are equivalent, by looking at all the artists_name inside each id and finding the longest common substring for such artists so that they have the common name for all the songs. We have created an internal identifier to mark each unique artist name and now we want to join using the artist_id to the full dataset, and retrieving the fields "artist_name" and "artist_iid" for the df_consensus_artists_final:

In [394]:
# Get all the database for the RAW database
df = query_db(qq="SELECT * from youtube_url", to_df=True)
df.columns = ["track_id","title","artist_id","artist_name","yt_url","duration","year"]
connection.close()

Perform the merge:

In [420]:
df.head()

,track_id,title,artist_id,artist_name,yt_url,duration,year
0,TRMMMYQ128F932D901,Silent Night,ARYZTJS1187B98C555,Faster Pussy cat,,252.05506,2003
1,TRMMMKD128F425225D,Tanssi vaan,ARMVN3U1187FB3A1EB,Karkkiautomaatti,,156.55138,1995
2,TRMMMRX128F93187D9,No One Could Ever,ARGEKB01187FB50750,Hudson Mohawke,,138.97098,2006
3,TRMMMCH128F425532C,Si Vos Querés,ARNWYLR1187B9B2F9C,Yerba Brava,,145.05751,2003
4,TRMMMWA128F426B589,Tangle Of Aspens,AREQDTE1269FB37231,Der Mystic,,514.29832,0


In [421]:
df_consensus_artists_final.head()

,artist_id,artist_name,artist_iid
0,AR002UA1187B9A637D,The Bristols,0
1,AR006821187FB5192B,Stephen Varcoe,1
2,AR009211187B989185,Carroll Thompson,2
3,AR009SZ1187B9A73F4,Gorodisch,3
4,AR00A1N1187FB484EB,1.000 Mexicans,4


In [422]:
df_final = pd.merge(df[["track_id","title","artist_id","yt_url","duration","year"]],
                   df_consensus_artists_final,
                   on = ["artist_id"],
                   how = "inner")

Unique artist names:

In [425]:
len(set(df_final.artist_name))

35678

Unique artist id:

In [426]:
len(set(df_final.artist_id))

36180

Unique artist iid:

In [427]:
len(set(df_final.artist_iid))

35678

Head:

In [424]:
df_final.head()

,track_id,title,artist_id,yt_url,duration,year,artist_name,artist_iid
0,TRMMMKD128F425225D,Tanssi vaan,ARMVN3U1187FB3A1EB,,156.55138,1995,Karkkiautomaatti,21822
1,TRMLAVN128F4252261,Yeah yeah Jenni,ARMVN3U1187FB3A1EB,,92.52526,1995,Karkkiautomaatti,21822
2,TRWUBYW128F4252258,Äl-oo-vee,ARMVN3U1187FB3A1EB,,76.77342,1995,Karkkiautomaatti,21822
3,TRGBNVG128F425224D,Jää beibi jää,ARMVN3U1187FB3A1EB,,107.59791,1995,Karkkiautomaatti,21822
4,TRBZRME128F425225E,Takaisin en tuu,ARMVN3U1187FB3A1EB,,80.61342,1995,Karkkiautomaatti,21822


## 3.1 Create a new database with the clean data

In [433]:
## 1.1 Connect to the database CLEAN

#Paths
path_db_final = os.path.join("..","data","MSD","clean.db")
path_sql_connection_db =  'sqlite:///' + path_db_final

#Connect
engine = db.create_engine(path_sql_connection_db)
connection = engine.connect()

In [437]:
df_final.to_sql('youtube_url', con = engine, index = False)

In [438]:
connection.close()

In [439]:
df_final.shape

(700382, 8)